# GeoJSON and Outline Features

In [ ]:
import dash
import dash_leaflet as dl
from jupyter_dash import JupyterDash
import random

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
## Data Import From Last Lesson
import geopandas as gpd
import json
zipfile = "zip://data/cb_2018_us_state_20m.zip"
states = gpd.read_file(zipfile)
zipfile = "zip://data/cb_2018_us_county_20m.zip"
counties = gpd.read_file(zipfile)

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

## Basic customizations
 * `color` - border color
 * `fillColor` - content color
 * `weight` - border thickness
 * `opacity` - border opacity
 * `fillOpacity` - content opacity
 * `dashArray`, `dashOffset` - governs dashed boundary pattern

These stylings should us CSS syntax

Use `options` property to change static colors, use a `hover_style` function to change the property

## When last we left you
### Exercise 1: Customizing the Outlines

In [ ]:
layer = dl.GeoJSON(data=json.loads(states.to_json()))
app = JupyterDash()
app.layout = dl.Map([layer, dl.TileLayer()],
                    center=[39, -98],
                    zoom=4,
                    style={
                        'width': '1000px',
                        'height': '500px'
                    })

app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Seque: A Javascript Digression
Several properties in the dash leaflet components require specifying a javascript function as the property. The `dash_extensions.javascript` module supplies three methods:
 
 * `arrow_function` - returns an fixed object. Useful if you want a function to return the same thing all the time
 * `assign` - allows you to inline javascript as a string
 * `Namespace` specifies a javascript namespace where the function is encapsulated. More on this one later.
 
#### Why is this important?
The property `hoverStyle` takes a **javascript function**. The simplest example is the use of the `arrow_function`


### Exercise 2: Adding Hover Behaviour

In [ ]:
from dash_extensions.javascript import arrow_function
hover_style = dict(weight=6)
layer = dl.GeoJSON(data=json.loads(states.to_json()),
                   hoverStyle=arrow_function(hover_style))
app = JupyterDash()
app.layout = dl.Map([layer, dl.TileLayer()],
                    center=[39, -98],
                    zoom=4,
                    style={
                        'width': '1000px',
                        'height': '500px'
                    })

app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Exercise 3: Tooltips
Tooltips are easy and a little strangely implemented. If the `data` has a column named `tooltip` that will be the tool tip. Bear in mind it can contain html.

In [ ]:
states_with_tooltip = states.copy(deep=True)
states_with_tooltip['tooltip'] = states.NAME
layer = dl.GeoJSON(data=json.loads(states_with_tooltip.to_json()))
app = JupyterDash()
app.layout = dl.Map([layer, dl.TileLayer()],
                    center=[39, -98],
                    zoom=4,
                    style={
                        'width': '1000px',
                        'height': '500px'
                    })

app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Exercise 4: Data Filtering

In [ ]:
from dash import html
from dash.dependencies import Input, Output
layer = dl.GeoJSON(id='borders',
                   data=json.loads(states.to_json()),
                   zoomToBoundsOnClick=True)
app = JupyterDash()
app.layout = html.Div([
    dl.Map([dl.TileLayer(), layer],
           center=[39, -98],
           zoom=4,
           style={
               'width': '1000px',
               'height': '500px'
           }),
    html.Button("Click Me!", id='btn')
])
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

### Exercise 5: Zoom Control

In [ ]:
from dash import html
from dash.dependencies import Input, Output
california = counties[counties.STATEFP == '06']
layer = dl.GeoJSON(id='counties',
                   data=json.loads(california.to_json()),
                   zoomToBoundsOnClick=True,
                   zoomToBounds=False)
app = JupyterDash()
app.layout = html.Div([
    dl.Map([dl.TileLayer(), layer],
           center=[37, -119],
           zoom=5,
           style={
               'width': '1000px',
               'height': '500px'
           }),
    html.Button("Click Me!", id='btn')
])


@app.callback(Output('counties', 'data'), Input('btn', 'n_clicks'))
def sandiego(input_):
    if input_ and (input_ % 2 == 1):
        return json.loads(california[california.COUNTYFP == '073'].to_json())
    else:
        return json.loads(california.to_json())


app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

## Bonus: The Anti-meridian Problem
If you try the same exercise with Alaska, you'll notice the the zoom is not working as expected. That is because the county of Aleutians West County crosses the "Antimeridian" line (180$^\circ$ longitude). So some values might be -179 others might be 179 longitude. The way to correct it is to map 179 latitude to -181 latitude.

The following visualization illustrates the issue. Note that the FIPS code for Aleutians West is 016

In [ ]:
from dash import html
from dash.dependencies import Input, Output
alaska = counties[counties.STATEFP == '02']
# alaska = alaska[alaska.COUNTYFP!='016']
layer = dl.GeoJSON(id='counties',
                   data=json.loads(alaska.to_json()),
                   zoomToBoundsOnClick=True,
                   zoomToBounds=True)
app = JupyterDash()
app.layout = html.Div([
    dl.Map([dl.TileLayer(), layer],
           center=[39, -98],
           zoom=4,
           style={
               'width': '1000px',
               'height': '500px'
           }),
    html.Button("Click Me!", id='btn')
])


@app.callback(Output('counties', 'data'), Input('btn', 'n_clicks'))
def select_alaska(input_):
    if input_ and (input_ % 3 == 1):
        return json.loads(alaska[alaska.COUNTYFP == '068'].to_json())
    elif input_ and (input_ % 3 == 2):
        return json.loads(alaska[alaska.COUNTYFP == '016'].to_json())
    else:
        return json.loads(alaska.to_json())


app.run_server(mode='inline', port=random.choice(range(2000, 10000)))